Implementar uma rede MLP, em python, sem usar pacotes prontos (e.g., Pytorch, Tensorflow, etc.)
Com a rede implementada, desenvolver dois modelos: um para classificação e um para regressão.
Avaliar os hiperparâmetros dos modelos variando o número de camadas, número de neurônio e taxas (eta e momentum).

Importanto as bibliotecas necessárias

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Importando o dataset iris do scikit-learn

In [4]:
from sklearn.datasets import load_iris
iris_dataset = load_iris()

--------------------------------------------------------------------------Funções de ativação-----------------------------------------------------------------------------------

In [5]:
def linear(x): 
    return x 

In [ ]:
def degrau(x): 
    return np.where(x >= 0, 1, 0) 

A função degrau acima retorna 1 se x > 0 ou 0 caso contrario

In [ ]:
def sigmoid(x): 
    return 1 / (1 + np.exp(-x)) 

A função sigmoid logística acima retorna f(x) = 1 / (1 + exp(-x))

In [6]:
def tanh(x):
    return np.tanh(x)

In [ ]:
def relu(x): 
    return np.maximum(0, x)

A função relu acima retorna o máximo entre o X e 0

In [ ]:
def softmax(x): 
    exp = np.exp(x - np.max(x))
    return exp / np.sum(exp, axis=-1, keepdims=True)

A função softmax converte a saída padrão em uma distribuição de probabilidade, da forma: f(y) = exp(y) / somatorio de K exp(yk)

--------------------------------------------------------------------------Retropropagração-----------------------------------------------------------------------------------